### 测试

In [ ]:
from utils.dataset_utils import get_distribution, remove_class
from instance_selection.parameter.parameter import *  # 导入参数的设定
from instance_selection.operator.init_toolbox import init_toolbox_emosaic
from instance_selection.operator.metrics import calculate_gmean_mauc, calculate_average_gmean_mauc
from instance_selection.operator.genetic_operator import selTournamentNDCD
from instance_selection.operator.ensemble import vote_result_ensembles, ensemble_individuals
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import scipy.io as sio  # 从.mat文件中读取数据集
import random
from deap import tools
import warnings

warnings.filterwarnings("ignore")  # 忽略警告

datasetname = 'Chess.mat'
mat_data = sio.loadmat('../datasets/mat/' + datasetname)

x = mat_data['X']
y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
#x,y=remove_class(x,y,1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=RANDOM_SEED)  # 划分数据集
scaler = StandardScaler()  # 数据的标准化
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
unique_elements_all, classes_all, counts_all = get_distribution(y)  # 获取原始数据集分布
unique_elements_train, classes_train, counts_train = get_distribution(y_train)  # 获取训练集分布
unique_elements_test, classes_test, counts_test = get_distribution(y_test)  # 获取测试集分布
print(datasetname + f' distribution: {counts_all}')
print(f'trainset distribution: {counts_train}')
print(f'testset distribution: {counts_test}')

# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(y)
# from scipy.io import savemat
# # 保存为 .mat 文件
# data_dict = {'X': x, 'Y': y.reshape(-1, 1)}  # 以字典形式存储
# savemat('USPS.mat', data_dict)
# # 输出保存成功
# print("数据保存成功！")

In [1]:
import numpy as np

def ensure_min_samples(X, Y, ind, min_samples=5):
    """
    确保ind中选择的实例，每个类别至少有min_samples个，
    若不足，则从未选择的实例中补充。
    
    :param X: 实例数据，形状为 (n_samples, n_features)
    :param Y: 标签数据，形状为 (n_samples,)
    :param ind: 选择序列，形状为 (n_samples,)
    :param min_samples: 每个类别的最小选择样本数
    :return: 更新后的选择序列 ind
    """
    ind = np.array(ind)
    Y = np.array(Y)
    
    unique_classes = np.unique(Y)
    
    for cls in unique_classes:
        selected_indices = np.where((ind == 1) & (Y == cls))[0]
        unselected_indices = np.where((ind == 0) & (Y == cls))[0]
        
        if len(selected_indices) < min_samples:
            num_needed = min_samples - len(selected_indices)
            if len(unselected_indices) >= num_needed:
                additional_indices = np.random.choice(unselected_indices, num_needed, replace=False)
            else:
                additional_indices = unselected_indices  # 如果不足，则全选
            
            ind[additional_indices] = 1
    
    return ind

# 示例数据
X = np.random.rand(20, 5)  # 20个实例，每个实例5个特征
Y = np.random.randint(0, 4, 20)  # 20个实例的标签，类别为 0,1,2,3
ind = np.random.choice([0, 1], size=20, p=[0.7, 0.3])  # 随机选择部分实例

updated_ind = ensure_min_samples(X, Y, ind, min_samples=5)
print("更新后的选择序列:", updated_ind)


更新后的选择序列: [1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1]
